In [14]:
from music21 import converter, instrument, note, chord
import glob 
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [2]:
notes = []
for file in glob.glob("midi_songs/*.mid"):
    
    '''
    music21.converter : 다양한 음악 파일 포맷으로부터 음악을 로드하는 툴을 가지고 있습니다.
    music21.converter.parse() : 파일의 아이템을 파싱하고 스트림에 넣어줍니다.
    instrument.partitionByInstrument() : 단일 스트림, 스코어, 멀티 파트 구조인 경우에 각각의 악기별로 파티션을 나누고 다른 파트들을 하나로 합쳐줍니다.
    Stream.recurse : 스트림안에 존재하는 Music21 객체가 가지고 있는 값들의 리스트를 반복(iterate) 할 수 있는 iterator 를 리턴해줍니다.
    
    '''
    
    # music21 스트림 개체로 로드 
    # 노트 : 계이름, 옥타브,오프셋
    # 코드 : 동시에 연주되는 노트 세트를 위한 컨테이너 
    midi = converter.parse(file)
    notes_to_parse = None
    try:       # 학습 데이터 중 TypeError 를 일으키는 파일이 있어서 해놓은 예외처리
        
        # 어떤 악기가 들어가있는지 알면, 혼합된 소리 중에서 특정 악기만을 뽑아낼 수 있음
        # 출처 :http://web.mit.edu/music21/doc/moduleReference/moduleInstrument.html
        parts = instrument.partitionByInstrument(midi)
    except TypeError:
        print('## 1 {} file occur error.'.format(file))
    if parts: # file has instrument parts
        print('## 2 {} file has instrument parts'.format(file))
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        print('## 3 {} file has notes in a flat structure'.format(file))
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

## 2 midi_songs\0fithos.mid file has instrument parts
## 2 midi_songs\8.mid file has instrument parts
## 2 midi_songs\ahead_on_our_way_piano.mid file has instrument parts
## 2 midi_songs\AT.mid file has instrument parts
## 2 midi_songs\balamb.mid file has instrument parts
## 2 midi_songs\bcm.mid file has instrument parts
## 2 midi_songs\BlueStone_LastDungeon.mid file has instrument parts
## 2 midi_songs\braska.mid file has instrument parts
## 2 midi_songs\caitsith.mid file has instrument parts
## 2 midi_songs\Cids.mid file has instrument parts
## 2 midi_songs\cosmo.mid file has instrument parts
## 2 midi_songs\costadsol.mid file has instrument parts
## 2 midi_songs\dayafter.mid file has instrument parts
## 2 midi_songs\decisive.mid file has instrument parts
## 2 midi_songs\dontbeafraid.mid file has instrument parts
## 2 midi_songs\DOS.mid file has instrument parts
## 2 midi_songs\electric_de_chocobo.mid file has instrument parts
## 2 midi_songs\Eternal_Harvest.mid file has instrument p

In [19]:
# 다음 노트를 예측하기 위해서 도움이 되는 이전 100개이ㅡ 노트를 사용하겠다
sequence_length = 100
n_vocab=358
# 모든 계이름의 이름을 pitchnames 변수에 저장.
# set 으로 중복을 피하고, sorted 함수로 정렬함.
pitchnames = sorted(set(item for item in notes))

# 각 계이름을 숫자로 바꾸는 dictionary(사전)을 만든다.
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []

# 입력 시퀀스를 만든다.
for i in range(0, len(notes) - sequence_length, 1):
    
    sequence_in = notes[i:i + sequence_length]
    # 바로 다음 term의 노트들을 넣어줌
    sequence_out = notes[i + sequence_length]
    # 노트 -> 숫자 matrix에서 꺼내옴
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

# 데이터 입력 형태를 LSTM 레이어에 알맞게 변경함.
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

# 입력값을 normalizing(정규화)
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [17]:
model = Sequential()
model.add(LSTM(256,
               input_shape=(network_input.shape[1], network_input.shape[2]),
               return_sequences=True))\

model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss',
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)

# Generating Music

In [ ]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# 각각의 뉴런(노드)의 가중치를 로드합니다.
# 파일에 저장한 학습 결과를 가져오는 것과 같습니다!
model.load_weights('weights.hdf5')

In [ ]:
# 입력 시퀀스를 랜덤하게 주는 부분.
start = numpy.random.randint(0, len(network_input)-1)

# 숫자를 노트로 매핑하는 사전을 생성합니다.
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]

prediction_output = []

# 500 개의 노트를 만들어줍니다.
for note_index in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    # 입력 값에 대해 다음 노트를 예측합니다.
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)

    # 결과값은 숫자가 아닌 노트여야 하므로, 미리 만들어놓은 사전에 숫자를 넣어서 맵핑시킵니다.
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [ ]:
ffset = 0
output_notes = []
# 모델에 의해 예측된 값을 바탕으로 노트와 코드(chord) 객체를 만듭니다.
for pattern in prediction_output:
    # 패턴(출력값)이 코드(chord) 일 때
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # 패턴이(출력값이) 노트일 때
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # 각 반복마 오프셋을 0.5 씩 증가시켜 줍니다.
    # 그렇지 않으면 같은 오프셋에 음이 쌓이게 됩니다.
    offset += 0.5